# The distillation

Welcome back! You already know how to use simple Datalog queries and stored relations in Cozo, and you have learned the intricacies of schema-based triple stores. Today we are going to learn about aggregations and algorithms.

Before we start, we need to get some data into the database so that we can play with them. Instead of sesame-seed-sized inline data we used the last few times, today we are moving towards peanut-sized data. The data we are going to use, and many examples that we will present, are adapted from the book [Practical Gremlin](https://kelvinlawrence.net/book/Gremlin-Graph-Guide.html), which teaches the Gremlin graph query language, a very different, imperative take on graphs (Datalog, by constrast, is declarative). It is always a good idea to explore different options for your problem and to decide for yourself which tool is best for you.

We start by defining the schema we need:

In [2]:
:schema

put country {
    code: string unique,
    desc: string
}

put continent {
    code: string unique,
    desc: string
}

put airport {
    iata: string unique,
    icao: string index,
    city: string index,
    desc: string,
    region: string index,
    country: ref,
    runways: int,
    longest: int,
    altitude: int,
    lat: float,
    lon: float
}

put route {
    src: ref,
    dst: ref,
    distance: int
}

put geo {
    contains: ref many,
}

eval::attr_name_conflict 

 × Attribute name country.code conflicts with an existing name

We intend the entities to be countries, continents, airports and routes. The attribute `geo.contains` denotes geographical inclusion. In our case, the `src` and `dst` of a `route` are always airport entities. Airports are uniquely identified by their `iata` code, and contain a slew of other attributes including latitudes and longitudes.

For the actual data, there is a problem: though peanut-sized by today's standard, the data still contains over 61k lines of JSON objects, some of which are quite long lines (yes, each line in the tx script is a valid JSON object). If you enter that into the JupyterLite web interface, your browser will freeze (mine did, anyway). That's why, for the playground, Cozo has the ability to execute scripts on your local file system. This sounds like a big security hole and it is, so it is disabled outside the playground.

Now download the data, look over it to see what it contains, put it somewhere on your hard drive (we recommend next to the `cozoserver` executable so that the following script works verbatim) and run:

In [5]:
:db execute '../tests/air-routes-data.json'

asserts,retracts
197646,0


The execution should not take to long. When it's done, we are set.

## Exploratory data analysis

The data is new to us. First we need to see what it looks like. Let's start with airports.

In [8]:
?[iata, city, desc, region, runways, lat, lon] := 
    [a airport.iata iata],
    [a airport.city city],
    [a airport.desc desc],
    [a airport.region region],
    [a airport.runways runways],
    [a airport.lat lat],
    [a airport.lon lon]
    
:limit 5

iata,city,desc,region,runways,lat,lon
ANC,Anchorage,Anchorage Ted Stevens,US-AK,3,61.1744003295898,-149.996002197266
ATL,Atlanta,Hartsfield - Jackson Atlanta International Airport,US-GA,5,33.6366996765137,-84.4281005859375
AUS,Austin,Austin Bergstrom International Airport,US-TX,2,30.1944999694824,-97.6698989868164
BNA,Nashville,Nashville International Airport,US-TN,4,36.1245002746582,-86.6781997680664
BOS,Boston,Boston Logan,US-MA,6,42.36429977,-71.00520325


The only notable thing about this query is that we used the `:limit` option to limit the number of output rows. If we did not put it in, thousands of rows will be returned and your browser may not like it. The `:offset` option is also available:

In [25]:
?[iata, city, desc, region, runways, lat, lon] := 
    [a airport.iata iata],
    [a airport.city city],
    [a airport.desc desc],
    [a airport.region region],
    [a airport.runways runways],
    [a airport.lat lat],
    [a airport.lon lon]

:offset 3
:limit 2

iata,city,desc,region,runways,lat,lon
BNA,Nashville,Nashville International Airport,US-TN,4,36.1245002746582,-86.6781997680664
BOS,Boston,Boston Logan,US-MA,6,42.36429977,-71.00520325


There is a subtle point here: when you specify `:limit`, the database is constrained to return only that many rows to you. But _which_ rows it gives you is not specified (for performance reasons). In our case, even though the first returned IATA is ANC, that doesn't mean the smallest IATA is ANC (the output is sorted, yes, but only among the rows themselves). In fact, the query didn't even look at all the rows, since it can already satisfy what you ask it for by looking only at five rows!

If you want "global" sorting for your results before applying `:limit`, you have to ask for it and the database will be forced to look at all the data:

In [27]:
?[iata, city, desc, region, runways, lat, lon] := 
    [a airport.iata iata],
    [a airport.city city],
    [a airport.desc desc],
    [a airport.region region],
    [a airport.runways runways],
    [a airport.lat lat],
    [a airport.lon lon]
    
:limit 5
:order iata

iata,city,desc,region,runways,lat,lon
AAA,Anaa,Anaa Airport,PF-U-A,1,-17.3526000976562,-145.509994506836
AAE,Annabah,Annaba Airport,DZ-36,2,36.8222007751465,7.80916976928711
AAL,Aalborg,Aalborg Airport,DK-81,2,57.0927589138,9.84924316406
AAN,Al Ain,Al Ain International Airport,AE-AZ,1,24.2616996765137,55.6091995239258
AAQ,Anapa,Anapa Airport,RU-KDA,1,45.0021018981934,37.3473014831543


You can also sort in descending order (by prefixing the sorted column name by the minus sign), or sort by multiple columns:

In [29]:
?[iata, city, desc, region, runways, lat, lon] := 
    [a airport.iata iata],
    [a airport.city city],
    [a airport.desc desc],
    [a airport.region region],
    [a airport.runways runways],
    [a airport.lat lat],
    [a airport.lon lon]
    
:limit 10
:order -runways, -city

iata,city,desc,region,runways,lat,lon
DFW,Dallas,Dallas/Fort Worth International Airport,US-TX,7,32.896800994873,-97.0380020141602
ORD,Chicago,Chicago O'Hare International Airport,US-IL,7,41.97859955,-87.90480042
DTW,Detroit,"Detroit Metropolitan, Wayne County",US-MI,6,42.2123985290527,-83.353401184082
DEN,Denver,Denver International Airport,US-CO,6,39.8616981506348,-104.672996520996
BOS,Boston,Boston Logan,US-MA,6,42.36429977,-71.00520325
AMS,Amsterdam,Amsterdam Airport Schiphol,NL-NH,6,52.3086013794,4.76388978958
UFA,Ufa,Ufa International Airport,RU-BA,5,54.5574989318848,55.8744010925293
YYZ,Toronto,Toronto Pearson International Airport,CA-ON,5,43.6772003174,-79.6305999756
TRG,Tauranga,Tauranga Airport,NZ-BOP,5,-37.6719017028809,176.195999145508
SNN,Shannon,Shannon Airport,IE-CE,5,52.7019996643066,-8.92481994628906


The above query finds the airports with the most runways, sorted by their city in reverse alphabetical order.

Of course, the first question when we have new data is "how many rows". We delayed answering this question since it requires aggregation (technically you can do it with aggregation since the query language we learned in the first tutorial is already Turing complete. But you need to get back lots of irrelevant stuff together with the count if you do it that way. Turing machines are not efficient). Here it is, how to count:

In [31]:
?[count(ct)] := [ct airport.iata iata]

ct
3504


The body of the rule is simple: we asked for all triples with the unique attribute `airport.iata`. But the aggregation `count` is applied to the _head_ of the rule instead of within the rule body.

We can mix aggregated head symbols with non-aggregates:

In [34]:
?[count(ct), initial] := [ct airport.iata iata], initial <- first(chars(iata))

:order -ct

ct,initial
245,S
235,B
228,M
214,C
212,A
211,Y
205,T
203,P
197,K
184,L


This gives you the number of airports with different initials. Any non-aggregated symbols in the head acts as grouping variables (similar to `group by` in SQL).

Another caveat lies here. Usually you can break a rule body into smaller parts by introducing other rules. But if we naively try to "refactor" the above query, we get nonsensical results:

In [38]:
initials[i] := [_ airport.iata iata], i <- first(chars(iata))
?[count(ct), initial] := initials[initial], ct <- initial

:order -ct

ct,initial
1,A
1,B
1,C
1,D
1,E
1,F
1,G
1,H
1,I
1,J


What's happening? Remember that Cozo Datalog operates with set semantics instead of bag semantics. So in the first rule, the results are already de-duplicated. But for aggregations like `count`, counting must be done with bag semantics. In fact, if the first rule can _disambiguate_ the duplicates, you get the old results:

In [39]:
initials[i, iata] := [_ airport.iata iata], i <- first(chars(iata))
?[count(ct), initial] := initials[initial, _], ct <- initial

:order -ct

ct,initial
245,S
235,B
228,M
214,C
212,A
211,Y
205,T
203,P
197,K
184,L


There are many aggregate functions in Cozo, most of them should be quite familiar for anyone fluent in SQL. For example, the following calculates the statistics for runways:

In [52]:
?[mean(runways), std_dev(runways)] := [a airport.runways runways]

:order mean_runways

parser::sort_key_not_found 

 × Sort key 'mean_runways' not found
 ╭─[2:1]
 2 │ 
 3 │ :order mean_runways
 · ──────────── 
 ╰────